#1.라이브러리 불러오기

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series
import plotly.offline as pyo
import plotly.graph_objs as go
import zipfile
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
import datetime
from sklearn.metrics import mean_absolute_error

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

plt.rc("font", family="Malgun Gothic") # window
plt.rc("axes", unicode_minus=False) # x,y축 (-)부호 표시

#2.데이터 불러오기

In [ ]:
train

##1.train nan 값 처리

In [ ]:
train.isnull().sum()

datetime               0
4단지배수지 공업 유출유량 적산차    12
dtype: int64

In [ ]:
train = train.fillna(method = 'ffill') #이전행으로

In [ ]:
train.isnull().sum()

datetime              0
4단지배수지 공업 유출유량 적산차    0
dtype: int64

In [ ]:
test 

In [ ]:
test.isnull().sum()

datetime              0
4단지배수지 공업 유출유량 적산차    0
dtype: int64

In [ ]:
result


#3.이상치 제거하기

In [ ]:
from sklearn.preprocessing import minmax_scale

In [ ]:
train.cpumns = [['datetime', 'y']]
test.columns = [['datetime', 'y']]

In [ ]:
#이상치 제거 해야지

def dr_outlier(df):
    quartile_1 = df.quantile(0.25)
    quartile_3 = df.quantile(0.75)
    IQR = quartile_3 - quartile_1

    condition = (df < (quartile_1 - 1.5 * IQR)) | (df > (quartile_3 + 1.5 * IQR))
    df[condition]=np.mean(df)
    

    # df[condition] = minmax_scale(df)

    # for i in range(len(df)):
    #   if (df[i] < (quartile_1 - 1.5 * IQR)) | (df[i] > (quartile_3 + 1.5 * IQR)):
    #     df[i] = df[i-1]
    return df


In [ ]:
train['y'] = dr_outlier(train['y'])

<ipython-input-13-aff5a8001096>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import keras 
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

import tensorflow as tf

##1.정규화하기

In [ ]:
df_all = pd.concat([train, test])
df_all.index = df_all['datetime']
df_all = df_all.drop(['datetime'], axis = 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import RobustScaler
scaler = MinMaxScaler()
# scaler = RobustScaler()
df_y = scaler.fit_transform(df_all)

In [ ]:
df_all['y'] = df_y
df_all = df_all.reset_index(drop = False)

In [ ]:
idx_train = len(train)
idx_test = len(test)

In [ ]:
train = df_all.iloc[:idx_train]
test = df_all.iloc[idx_train:]

In [ ]:
idx = []
for i in range(len(test)):
  idx.append(i)
test.index = idx

#4.예측(2021-12-31 23:0:00)

##1.모델 학습(1시간만 먼저 예측해서 학습)

In [ ]:
import datetime

In [ ]:
def add_next_date(df):
  lst_next_date = []
  current = datetime.datetime.strptime(df.iloc[-1]['datetime'], "%Y-%m-%d %H:%M:%S")
  next_date = current + datetime.timedelta(hours=1)
  lst_next_date.append(next_date)
  
  plus_time = pd.DataFrame(lst_next_date, columns=["datetime"])
  plus_time

  df = pd.concat([df, plus_time])
  df["datetime"] = df["datetime"].astype(str)
  df.reset_index(drop=True, inplace=True)
  df = df.fillna(0)
  return df

In [ ]:
def add_cloumns(df):
    df = df.astype({'datetime' : str})
    df['year'] = [date.split('-')[0] for date in df['datetime']]
    df['month'] = [date.split('-')[1] for date in df['datetime']]
    df['day'] = [date.split('-')[2] for date in df['datetime']]
    df['day'] = [date.split(' ')[0] for date in df['day']]
    df['hour'] = [date.split(' ')[-1] for date in df['datetime']]
    df['hour'] = [date.split(':')[0] for date in df['hour']]
    df=df.astype({'year':"int",'month':"int",'day':"int",'hour':"int"})
    return df

In [ ]:
#모델 Conv1D할 때만 필요
def data_reshape(df):
  df = np.array(df)
  df = np.reshape(df, (len(df), 4, 1))
  return df

In [ ]:
test = add_next_date(test)

train_data=add_cloumns(train)
test_data=add_cloumns(test)

x_train=train_data[['year','month','day','hour']]
y_train=train_data['y']

x_test=test_data[['year','month','day','hour']]
y_test=test_data['y']

#모델 Conv1D할 때만 필요
x_train = data_reshape(x_train)
x_test = data_reshape(x_test)

In [ ]:
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
#모델은 잘 나온걸로 학습
#모델 구성
model = keras.Sequential()
model.add(Conv1D(4, 3, activation = 'relu', input_shape = (4, 1)))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
# model.add(Dense(64, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1))

#모델 학습과정 설정
optimizers=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=optimizers, loss="mse", metrics="mae")


early_stopping = EarlyStopping(monitor="val_loss",
                               patience=50)
#모델 학습
model.fit(x_train,y_train,epochs=20,batch_size=32,callbacks=early_stopping,validation_data=(x_test, y_test))

In [ ]:
#모델 평가
test_mse_score, test_mae_score = model.evaluate(x_test, y_test)
print(test_mae_score)
print(test_mse_score)

predictions = model.predict(x_test)

264/264 [==============================] - 1s 2ms/step - loss: 0.0131 - mae: 0.0881
0.08808225393295288
0.013063675723969936
264/264 [==============================] - 1s 2ms/step


In [ ]:
#파일명은 자신이 선정한 모델로 바꾸기
from keras.models import load_model
model.save('./ConvD1_model.h5')

In [ ]:
test['y'].iloc[-1] = predictions[-1]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



##2.모델 불러와서 나머지 예측

In [ ]:
a = test.copy()

- 5~10분 정도 걸려요!!!

In [ ]:
for i in tqdm(range(335)):
  a = add_next_date(a)

  train_data=add_cloumns(train)
  test_data=add_cloumns(a)

  x_train=train_data[['year','month','day','hour']]
  y_train=train_data['y']

  x_test=test_data[['year','month','day','hour']]
  y_test=test_data['y']

  #모델 Conv1D할 때만 필요
  x_train = data_reshape(x_train)
  x_test = data_reshape(x_test)

  from keras.models import load_model
  new_model = load_model('./ConvD1_model.h5') #내가 저장한 파일로 불러오기

  predictions = new_model.predict(x_test)


  a.iloc[[-1], [1]] = predictions[-1]

#5.전체 예측

In [ ]:
train_data=add_cloumns(train)
test_data=add_cloumns(a)

x_train=train_data[['year','month','day','hour']]
y_train=train_data['y']

x_test=test_data[['year','month','day','hour']]
y_test=test_data['y']

x_train = np.array(x_train)
x_train = np.reshape(x_train, (len(x_train), 4, 1))

x_test = np.array(x_test)
x_test = np.reshape(x_test, (len(x_test), 4, 1))

model = load_model('./ConvD1_model.h5') #내가 저장한 파일로 불러오기

test_mse_score, test_mae_score = model.evaluate(x_test, y_test)

predictions = model.predict(x_test)

274/274 [==============================] - 1s 2ms/step


#6.결과 집어 넣기

##1.역전규화

In [ ]:
predictions = scaler.inverse_transform(predictions)
predictions

array([[433.04794],
       [433.08008],
       [433.11215],
       ...,
       [451.98196],
       [452.0088 ],
       [452.03558]], dtype=float32)

In [ ]:
len(predictions)

8760

In [ ]:
def result_csv(df):
  df_y_pred = pd.DataFrame(data = df, columns=["y_pred"])

  lst_data = []

  for i in range(0, len(df)-335):
      data = df_y_pred["y_pred"].loc[i:i+335].values.tolist()
      lst_data.append(data) 

  df_submission = pd.DataFrame(data=lst_data, columns=result.columns[1:])

  file = pd.concat([result["datetime"], df_submission], axis=1)
  file.columns = result.columns

  evaluation2 = file.loc[:8424]

  mission_result=evaluation2[evaluation2.columns[:]]
  mission_result.to_csv('task3.csv',encoding='CP949',index=False)

  return mission_result

In [ ]:
result_csv(predictions)